In [ ]:
import numpy as np
import torch
import json
import sys
import pandas as pd 
import os 
import numpy as np
import random
import unidecode 
from scipy.stats import kendalltau, spearmanr
from torch import cuda
from torch.utils.data import Dataset, DataLoader
from numpy.linalg import norm
from numpy import dot
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModel


In [ ]:
from indicnlp.normalize import indic_normalize
from indicnlp.transliterate import unicode_transliterate
from indicnlp.tokenize import indic_tokenize
from sacremoses import MosesPunctNormalizer
from sacremoses import MosesTokenizer

In [ ]:
lang='te'

In [ ]:
languages_map = {
    'en': {"label": "English", 'id': 0},
    'hi': {"label": "Hindi", 'id': 1},
    'te': {"label": "Telugu", 'id': 2},
    'bn': {"label": "Bengali", 'id': 3},
    'pa': {"label": "Punjabi", 'id': 4},
    'or': {"label": "Odia", 'id': 6},
    'as': {"label": "Assamese", 'id': 7},
    'gu': {"label": "Gujarati", 'id': 8},
    'mr': {"label": "Marathi", 'id': 9},
    'kn': {"label": "Kannada", 'id': 10},
    'ta': {"label": "Tamil", 'id': 11},
    'ml': {"label": "Malayalam", 'id': 12}
}


def get_language_normalizer():
    lang_normalizer = {}
    for k in languages_map:
        if k in ['ur']:
            continue
        elif k == 'en':
            lang_normalizer[k] = MosesPunctNormalizer()
        else:
            normfactory = indic_normalize.IndicNormalizerFactory()
            lang_normalizer[k] = normfactory.get_normalizer(k)
    return lang_normalizer
normalizer = get_language_normalizer()
en_tok = MosesTokenizer(lang="en")
def get_text_in_unified_script(text, lang):
    return unicode_transliterate.UnicodeIndicTransliterator.transliterate(
                " ".join(
                    indic_tokenize.trivial_tokenize(
                        normalizer[lang].normalize(text.strip()), lang
                    )
                ),
                lang,
                "hi",
            ).replace(" ् ", "्")


def process_text(text, lang):
    """ normalize and tokenize and then space join the text """
    if lang == 'en':
        return " ".join(en_tok.tokenize(normalizer[lang].normalize(text.strip()), escape=False)).strip()
    else:
        # return unified script text
        return get_text_in_unified_script(text, lang)

    
    



In [ ]:
def load_jsonl(file_path):
    res = []
    with open(file_path, encoding='utf-8') as dfile:
        for line in dfile.readlines():
            curr_dict = json.loads(line.strip())
            curr_dict['sentence'] = process_text(curr_dict['sentence'], lang)
            res.append(curr_dict)
    return res


filename = "./plan_order/{lang}/{lang}_order.jsonl".format(lang=lang)

reader  = load_jsonl(filename)


jsonl_data = pd.DataFrame(reader)
jsonl_data

In [ ]:
device = "cuda" if cuda.is_available() else "cpu"


class JsonL(Dataset):
    def __init__(self, dataframe, tokenizer, max_len, isTarget = True):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.targ = isTarget

    def __getitem__(self, index):

        sent  = self.data.sentence[index]
        factlist = self.data.facts[index]
        entity = self.data.entity_name[index]
        section = self.data.native_sentence_section[index]
    
        fact_str = process_facts(factlist,entity,section,lang)
        
        
        
        if self.targ :
            ref_order = self.data.order[index] 

            return {
                "fact_str" : fact_str,
                "sent" : sent,
                "ref_order" : ref_order
            }
        else :
            return {
                "fact_str" : fact_str,
                "sent" : sent
            }
            
            

    def __len__(self):
        return self.len





In [ ]:
def collate(batch):
    
    fact_str_list = []
    sent_list = []
    ref_list = []
    
    ref_flag = True 
    
    try :
        k = batch[0]['ref_order']
    except:
        ref_flag = False
    
    for i in batch :
        fact_str_list.append(i['fact_str'])
        sent_list.append(i['sent'])
        
        
        if ref_flag :
            ref_list.append(i['ref_order'])
        
    
    if ref_flag :
        return {'fact_str':fact_str_list,'sent' : sent_list,'ref_order':ref_list}
    else:
        return {'fact_str':fact_str_list,'sent' : sent_list}
    
    

In [ ]:
model_name = "google/mt5-small" #google/muril-base-cased, bert-base-multilingual-cased, xlm-roberta-base, sentence-transformers/LaBSE"
tokenizer = AutoTokenizer.from_pretrained(model_name,cache_dir="/scratch")


new_tokens = ['<H>', '<R>', '<T>', '<QR>', '<QT>', '<S>']
new_tokens_vocab = {}
new_tokens_vocab['additional_special_tokens'] = []
for idx, t in enumerate(new_tokens):
    new_tokens_vocab['additional_special_tokens'].append(t)
num_added_toks = tokenizer.add_special_tokens(new_tokens_vocab)
# self.config_args.logger.critical('added %s tokens' % num_added_toks)

In [ ]:
model_name = "google/mt5-small" #google/muril-base-cased, bert-base-multilingual-cased, xlm-roberta-base, sentence-transformers/LaBSE
PATH = "/scratch/model.ckpt" 

model = AutoModel.from_pretrained(model_name,output_hidden_states=True, cache_dir="/scratch")

model.resize_token_embeddings(len(tokenizer))

checkpoint = torch.load(PATH)
print (checkpoint.keys())

new_state_dict = {}
for ij in checkpoint['state_dict']:
    #     print (ij[:6])
    if ij[:6] == 'model.':
        newkey = ij[6:]
        new_state_dict[newkey] = checkpoint['state_dict'][ij]
    else :
        print (ij)
    
del new_state_dict['lm_head.weight']
model.load_state_dict(new_state_dict)

In [ ]:

languages_map = {
    'en': {"label": "English", 'id': 0},
    'hi': {"label": "Hindi", 'id': 1},
    'te': {"label": "Telugu", 'id': 2},
    'bn': {"label": "Bengali", 'id': 3},
    'pa': {"label": "Punjabi", 'id': 4},
    'ur': {"label": "Urdu", 'id': 5},
    'or': {"label": "Odia", 'id': 6},
    'as': {"label": "Assamese", 'id': 7},
    'gu': {"label": "Gujarati", 'id': 8},
    'mr': {"label": "Marathi", 'id': 9},
    'kn': {"label": "Kannada", 'id': 10},
    'ta': {"label": "Tamil", 'id': 11},
    'ml': {"label": "Malayalam", 'id': 12}
}


def get_nodes(n):
    n = n.strip()
    n = n.replace('(', '')
    n = n.replace('\"', '')
    n = n.replace(')', '')
    n = n.replace(',', ' ')
    n = n.replace('_', ' ')
    n = unidecode.unidecode(n)
    return n


def get_relation(n):
    n = n.replace('(', '')
    n = n.replace(')', '')
    n = n.strip()
    n = n.split()
    n = "_".join(n)
    return n


def linear_fact_str(fact, enable_qualifiers=True):
    fact_str = ['<R>', get_relation(fact[0]).lower(), '<T>', get_nodes(fact[1]).lower()]
    qualifier_str = [' '.join(['<QR>', get_relation(x[0]).lower(), '<QT>', get_nodes(x[1]).lower()]) for x in fact[2]]
    if enable_qualifiers and len(fact[2])>0:
        fact_str += [' '.join(qualifier_str)]
    return fact_str

def process_facts(facts, entity, section, language):
    """ linearizes the facts on the encoder side """
    linearized_facts = []
    for i in range(len(facts)):
        linearized_facts += linear_fact_str(facts[i], enable_qualifiers=True)
    processed_facts_str = ' '.join(linearized_facts)
    pre_string = "<H> %s %s <S> %s" % (entity.lower(), processed_facts_str, section)
#     pre_string = "<H> %s %s " % (entity.lower(), processed_facts_str)
    return "generate %s : %s" % (languages_map[language]['label'].lower(), pre_string)



def process_facts_a(facts, entity):
    """ linearizes the facts on the encoder side """
    triples = []
    subject = entity
    # triples.append([' <S> '+subject])

    for triple in facts:
        predicate = triple[0]
        object = triple[1]
        triples.append([' S| '+subject, ' P| '+predicate, ' O| '+object])

        for quals in triple[2]:
            qual_subject = object
            qual_predicate = quals[0]
            qual_object = quals[1]
            triples.append([' S| '+qual_subject, ' P| '+qual_predicate, ' O| '+qual_object])

    return triples


In [ ]:
BATCH_SIZE = 32
from tqdm import tqdm

ref_order = []
pred_order = []

data_set =JsonL(jsonl_data,tokenizer,518)
load_params = {"batch_size": BATCH_SIZE, "shuffle": False, "num_workers": 0}
data_loader = DataLoader(data_set, **load_params, collate_fn = collate)


for _, data in enumerate(data_loader, 0):
#     print (data)
    
    sent = data['sent']
    fact_str = data['fact_str']
    
    ref_order += data['ref_order']
    
    print ("sent ",len(sent))
    print ("fact ",len(fact_str))

#     print (sent)
    encoded_sentence = tokenizer.batch_encode_plus(sent, return_tensors="pt",padding=True)
    encoded_facts = tokenizer.batch_encode_plus(fact_str, return_tensors="pt",padding=True)
    
    model.to(device)
    with torch.no_grad():
        output = model(
            input_ids=encoded_facts["input_ids"].to(device,dtype=torch.long),
            attention_mask=encoded_facts["attention_mask"].to(device,dtype=torch.long),
            decoder_input_ids=encoded_sentence["input_ids"].to(device,dtype=torch.long),
            decoder_attention_mask=encoded_sentence["attention_mask"].to(device,dtype=torch.long),
            output_attentions=True  
        )
    
#     print (output.cross_attentions)
    print (output.cross_attentions[-1].shape)
    batch_fin_crossattention = output.cross_attentions[-1].mean(dim=1)
#     batch_fin_crossattention = output.cross_attentions[-1][:,2,:,:]
    print (batch_fin_crossattention.shape)
    print ("batch ",len(batch_fin_crossattention))
    
    for itr in range(len(batch_fin_crossattention)):
        
        fin_crossattention = batch_fin_crossattention[itr]
        print (fin_crossattention.shape)
    
        stflag = -1
        currfact = -1
        factpos = []
        factpos2 = []
        
        tokenized_sent = tokenizer.tokenize(sent[itr])
        tokenized_fact = tokenizer.tokenize(fact_str[itr])
        sent_len = len(tokenized_sent)
        for wordval in range(len(tokenized_fact)):

            tokenized_curr = tokenizer.tokenize(fact_str[itr])[wordval]
            if (tokenized_curr) == "<R>" and stflag == -1:
                stflag = 1

                sumval = 0 
                fcount = 0
                maxpos = 0
                maxscore = -99
                print ("FIRST FACT")

                continue

            elif (tokenized_curr) == "<R>" and stflag != -1:
                factpos.append(maxpos)
                factpos2.append(sumval/fcount)

                stflag = 1

                sumval = 0 
                fcount = 0
                maxpos = 0
                maxscore = -99
                print (len(factpos) , " FACT OVER")

                continue 


            elif (tokenized_curr) == "<QR>":
    #             stflag = 0
                print ("QR FOUNDS")
                continue

            elif (tokenized_curr) == "<S>":
                factpos.append(maxpos)
                factpos2.append(sumval/fcount)
                stflag = 0
                print (len(factpos) , "LAST FACT OVER")
                continue


#             print(fin_crossattention[:,wordval][:sent_len])
            max_fact = fin_crossattention[:,wordval][:sent_len].argmax()
            argval = fin_crossattention[:,wordval][:sent_len].max()
            try:
                print ("fact subword : ",tokenized_fact[wordval],end = "\t\t")
            except :
                print ("hello")
                pass
            
            intval = np.array(max_fact.to('cpu'))
            tempind = encoded_facts.word_ids()[intval]
            print ("matched sentence sub word : ",tokenized_sent[intval])

            if stflag == 1:
                sumval += (intval+1)
                fcount += 1
                if argval > maxscore:
                    maxpos = (intval+1)
                    maxscore = argval


        print ("===end===")
        final_ordering =  np.argsort(factpos2)
        pred_order.append(list(final_ordering))


        print ()
        print ("works")

    

In [ ]:
qidx = random.choice([z for z in range(100)])
print(pred_order[qidx])
print(ref_order[qidx])

In [ ]:
def get_accuracy(a, b):
    assert len(a)==len(b), "unequal size of list"
    count = 0
    for y, z in zip(a, b):
        if y==z:
            count+=1
    return count/len(a)

In [ ]:
avg_spearman = np.array([spearmanr(y, z)[0] for y, z in zip(pred_order, ref_order)])
avg_kendalltau = np.array([kendalltau(y, z)[0] for y, z in zip(pred_order, ref_order)])
print("spearman", avg_spearman.mean())
print("kendalltau", avg_kendalltau.mean())
print("accuracy", get_accuracy(ref_order, pred_order))
